In [8]:
# 구글 maps 예시

=== 목적지로 가는 경로 ===
걸어서 3분 이동 (거리 0.2 km)
서초중학교에서 버스 서울 간선버스을 타고 지하철2호선방배역까지 이동
출발 시간: 오전 10:28, 도착 시간: 오전 10:33, 정류장 수: 3
걸어서 2분 이동 (거리 0.1 km)
방배에서 지하철 서울 지하철 2호선을 타고 구로디지털단지까지 이동
출발 시간: 오전 10:38, 도착 시간: 오전 10:54, 정류장 수: 7
걸어서 3분 이동 (거리 0.2 km)

=== 출발지로 돌아오는 경로 ===
걸어서 3분 이동 (거리 0.2 km)
구로디지털단지에서 지하철 서울 지하철 2호선을 타고 방배까지 이동
출발 시간: 오전 10:31, 도착 시간: 오전 10:48, 정류장 수: 7
걸어서 2분 이동 (거리 0.1 km)
방배역.백석예술대학교에서 버스 서울 간선버스을 타고 서초중학교까지 이동
출발 시간: 오전 10:53, 도착 시간: 오전 10:59, 정류장 수: 3
걸어서 2분 이동 (거리 0.1 km)


In [11]:
import requests

def get_coordinates(client_id, client_secret, address):
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {
        'query': address
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['addresses']:
            x = data['addresses'][0]['x']  # 경도
            y = data['addresses'][0]['y']  # 위도
            return x, y
        else:
            print(f"주소를 찾을 수 없습니다: {address}")
            return None, None
    else:
        print("Failed to retrieve data:", response.status_code, response.text)
        return None, None

def get_driving_route(client_id, client_secret, start, goal):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret
    }
    params = {
        'start': f"{start[0]},{start[1]}",
        'goal': f"{goal[0]},{goal[1]}",
        'option': 'trafast'  # 최적 경로 옵션
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to retrieve data:", response.status_code, response.text)
        return None

def print_route_duration(route_info, direction):
    if route_info and 'route' in route_info and 'trafast' in route_info['route']:
        duration_ms = route_info['route']['trafast'][0]['summary']['duration']
        duration_seconds = duration_ms // 1000  # 밀리초를 초로 변환
        hours, remainder = divmod(duration_seconds, 3600)
        minutes, _ = divmod(remainder, 60)
        print(f"{direction} 이동 시간: {hours}시간 {minutes}분")
    else:
        print(f"{direction} 경로 정보를 가져올 수 없습니다.")

# 클라이언트 ID와 비밀번호
client_id = 'gx4wwzfwty'
client_secret = '71tBFr6FCb6qJPGzS5HjLekN8qhPIsBryYehi5d2'

# 주소를 입력하세요
start_address = '서울시 서초구 효령로 46길 21'
goal_address = '서울 구로구 디지털로32나길 22'

# 주소를 경도와 위도로 변환
start_coordinates = get_coordinates(client_id, client_secret, start_address)
goal_coordinates = get_coordinates(client_id, client_secret, goal_address)

if start_coordinates and goal_coordinates:
    # 출발지에서 도착지까지의 경로
    route_info_to = get_driving_route(client_id, client_secret, start_coordinates, goal_coordinates)
    print_route_duration(route_info_to, "출발지에서 도착지까지")

    # 도착지에서 출발지까지의 경로
    route_info_from = get_driving_route(client_id, client_secret, goal_coordinates, start_coordinates)
    print_route_duration(route_info_from, "도착지에서 출발지까지")


출발지에서 도착지까지 이동 시간: 0시간 28분
도착지에서 출발지까지 이동 시간: 0시간 37분
